In [8]:
import sqlite3

def get_match_ids_and_teams_for_player(player_name):
    conn = sqlite3.connect('ipl_database.db')
    cursor = conn.cursor()
    date_list = []
    team1_list = []
    cursor.execute("""
        SELECT ID,Date, Team1, Team2, Team1Players, Team2Players
        FROM ipl_match_list
        WHERE Team1Players LIKE ? OR Team2Players LIKE ?
    """, ('%' + player_name + '%', '%' + player_name + '%'))

    result = cursor.fetchall()

    print(len(result))
    for match_id, date, team1, team2, pl1, pl2 in result:
        date_list.append(date)
        
        if player_name in pl1:
            # print(f"Match ID: {match_id}, {player_name} is in {team1}, playing against {team2} on {date}")
            team1_list.append(team1)
        else:
            pass
            # print(f"Match ID: {match_id}, {player_name} is in {team2}, playing against {team1} on {date}")

    conn.close()
    return date_list, team1_list

In [9]:
player_name = 'DA Warner'
dt, t1l = get_match_ids_and_teams_for_player(player_name)

162


In [10]:
print(set(t1l))

{'Sunrisers Hyderabad', 'Delhi Daredevils', 'Delhi Capitals'}


In [11]:
sorted_dates = sorted(dt)

start_date = sorted_dates[0]  # Get the first element (oldest date)
end_date = sorted_dates[-1]  # Get the last element (newest date)

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2009-05-02
End Date: 2022-05-21


In [12]:
import search_results
search_dict = search_results.extract_elements()
player_list = search_dict["Players"]
print(player_list)
print(len(player_list))

['N Saini', 'Harpreet Brar', 'Shubman Gill', 'SS Tiwary', 'Mustafizur Rahman', 'PVD Chameera', 'GD Phillips', 'Anand Rajan', 'Gurkeerat Singh', 'BR Dunk', 'V Kohli', 'AB Agarkar', 'CK Langeveldt', 'KB Arun Karthik', 'SS Cottrell', 'N Jagadeesan', 'SPD Smith', 'DJ Jacobs', 'MS Bisla', 'A Ashish Reddy', 'L Balaji', 'MN van Wyk', 'CV Varun', 'K Yadav', 'RJ Peterson', 'DB Ravi Teja', 'Aman Hakim Khan', 'AJ Turner', 'C de Grandhomme', 'L Ngidi', 'VG Arora', 'KP Pietersen', 'B Sumanth', 'B Lee', 'SC Ganguly', 'CJ Dala', 'BCJ Cutting', 'A Mishra', 'PBB Rajapaksa', 'PD Collingwood', 'S Badrinath', 'M Vohra', 'Imran Tahir', 'Anureet Singh', 'RJ Harris', 'BA Bhatt', 'GC Viljoen', 'SE Marsh', 'CJ McKay', 'AC Gilchrist', 'SA Yadav', 'H Klaasen', 'KMA Paul', 'Bipul Sharma', 'M Prasidh Krishna', 'ST Jayasuriya', 'P Dogra', 'Iqbal Abdulla', 'KS Bharat', 'MJ Guptill', 'S Aravind', 'FH Edwards', 'Sunny Singh', 'PM Sarvesh Kumar', 'M Muralitharan', 'RT Ponting', 'V Sehwag', 'WP Saha', 'SP Goswami', 'D B

In [33]:
import sqlite3

def get_batting_positions_id_innings(match_id, innings):
    conn = sqlite3.connect('./ipl_database.db')
    cursor = conn.cursor()

    # SQL query to fetch all unique batters in the specified match ID and innings
    query = '''
    SELECT DISTINCT batter
    FROM ipl_ball_by_ball
    WHERE ID = ? AND innings = ?
    ORDER BY overs, ballnumber
    '''

    cursor.execute(query, (match_id, innings))
    batting_positions = [row[0] for row in cursor.fetchall()]

    conn.close()
    return batting_positions

# Example usage:
match_id = 1312200  # Example match ID
innings = 1  # Example innings
batting_positions = get_batting_positions_id_innings(match_id, innings)
print("Batting Positions:", batting_positions)

Batting Positions: ['YBK Jaiswal', 'JC Buttler', 'SV Samson', 'D Padikkal', 'SO Hetmyer', 'R Ashwin', 'R Parag', 'TA Boult', 'OC McCoy']


In [14]:
def classify_player(player_stats):
    innings_played = sum(player_stats['Innings'])  # Total number of innings played
    balls_bowled_per_match = player_stats['Balls'] / player_stats["Matches"]
    
    # Calculate the percentage of innings played at each batting position
    total_innings = sum(player_stats['Innings'])
    percentages = [innings / total_innings * 100 for innings in player_stats['Innings']]
    
    if percentages[0] + percentages[1] >= 80 and balls_bowled_per_match < 5:
        return "Opening Batsman"
    elif percentages[0] + percentages[1] + percentages[2] >= 70 and balls_bowled_per_match < 5:
        return "Top Order"
    elif percentages[2] + percentages[3] + percentages[4] >= 80 and balls_bowled_per_match < 5:
        return "Middle-order Batsman"
    elif percentages[4] + percentages[5] + percentages[6] >= 60 and balls_bowled_per_match < 5:
        return "Finisher"
    elif percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] >= 70 and balls_bowled_per_match > 15:
        return "All-Rounder"
    elif percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] >= 80 and 6 <= balls_bowled_per_match <= 15:
        return "Batting All-Rounder"
    elif (percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] + percentages[6] >= 65 or
          percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] + percentages[6] + percentages[7] >= 70) and balls_bowled_per_match > 15:
        return "Bowling All-Rounder"
    elif balls_bowled_per_match > 15:
        return "Bowler"
    elif percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] + percentages[6] >= 70:
        return "Batter"
    else:
        return "Player"

# Example usage:
player_stats = {
    'Innings': [190, 7, 300, 4, 6, 3, 2, 1, 0, 0, 0],  # Number of innings played at each batting position (1-11)
    'Balls': 1,  # Total balls bowled by the player
    'Matches': 555,
}

classification = classify_player(player_stats)
print("Player Classification:", classification)

Player Classification: Top Order


In [15]:
import data_manipulation
def fetch_player_stats(player_name):
    
    start_date, end_date = data_manipulation.get_first_and_last_match_dates(player_name)
    innings_played = data_manipulation.find_batting_positions(player_name)
    total_balls_bowled = data_manipulation.total_bowls(player_name, start_date, end_date)
    total_matches = data_manipulation.num_matches(player_name, start_date, end_date)
    print({'Innings': innings_played, 'Balls': total_balls_bowled, 'Matches': total_matches})
    return {'Innings': innings_played, 'Balls': total_balls_bowled, 'Matches': total_matches}

def classify_player(player_stats):
    balls_bowled_per_match = player_stats['Balls'] / player_stats["Matches"]
    
    # Calculate the percentage of innings played at each batting position
    total_innings = sum(player_stats['Innings'])
    percentages = [innings / total_innings * 100 for innings in player_stats['Innings']]
    
    if percentages[0] + percentages[1] >= 80 and balls_bowled_per_match < 5:
        return "Opening Batsman"
    elif percentages[0] + percentages[1] + percentages[2] >= 70 and balls_bowled_per_match < 5:
        return "Top Order"
    elif percentages[2] + percentages[3] + percentages[4] >= 80 and balls_bowled_per_match < 5:
        return "Middle-order Batsman"
    elif percentages[4] + percentages[5] + percentages[6] >= 60 and balls_bowled_per_match < 5:
        return "Finisher"
    elif percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] >= 70 and balls_bowled_per_match > 15:
        return "All-Rounder"
    elif percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] >= 80 and 6 <= balls_bowled_per_match <= 15:
        return "Batting All-Rounder"
    elif (percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] + percentages[6] >= 65 or
          percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] + percentages[6] + percentages[7] >= 70) and balls_bowled_per_match > 15:
        return "Bowling All-Rounder"
    elif balls_bowled_per_match > 15:
        return "Bowler"
    elif percentages[0] + percentages[1] + percentages[2] + percentages[3] + percentages[4] + percentages[5] + percentages[6] >= 70:
        return "Batter"
    else:
        return "Player"

# Fetch player stats for "DA Warner"
player_name = "SR Tendulkar"
player_stats = fetch_player_stats(player_name)

# Classify the player
classification = classify_player(player_stats)
print("Player Classification for", player_name + ":", classification)

78
{'Innings': [4, 65, 5, 4, 0, 0, 0, 0, 0, 0, 0], 'Balls': 37, 'Matches': 78}
Player Classification for SR Tendulkar: Opening Batsman


In [40]:
import sqlite3
import pandas as pd

def fetch_batting_scorecard(ID, innings):
    conn = sqlite3.connect('./ipl_database.db')

    # Fetch batting data
    batting_query = '''
    SELECT ball.batter, SUM(ball.batsman_run) AS Runs_Scored, COUNT(*) AS Balls_Faced, 
           SUM(CASE WHEN ball.batsman_run = 4 THEN 1 ELSE 0 END) AS Four_Count,
           SUM(CASE WHEN ball.batsman_run = 6 THEN 1 ELSE 0 END) AS Six_Count,
           SUM(CASE WHEN ball.player_out != 'not out' THEN 1 ELSE 0 END) AS Out, 
           CASE WHEN MAX(ball.isWicketDelivery) = 1 THEN MAX(ball.kind) ELSE 'not out' END AS Dismissal_Type, 
           MAX(ball.fielders_involved) AS fielders_involved, 
           MAX(CASE WHEN ball.player_out != 'not out' THEN ball.bowler ELSE NULL END) AS Bowler
    FROM ipl_ball_by_ball AS ball
    JOIN (
        SELECT DISTINCT batter
        FROM ipl_ball_by_ball
        WHERE ID = ? AND innings = ?
        ORDER BY overs, ballnumber
    ) AS batting_order ON ball.batter = batting_order.batter
    WHERE ball.ID = ? AND ball.innings = ?
    GROUP BY ball.batter
    ORDER BY MIN(ball.overs), MIN(ball.ballnumber);  -- Order by batting position
    '''

    batting_data = pd.read_sql_query(batting_query, conn, params=(ID, innings, ID, innings))

    conn.close()

    return batting_data


def fetch_bowling_scorecard(ID, innings):
    conn = sqlite3.connect('./ipl_database.db')

    # Fetch bowling data
    bowling_query = f'''
    SELECT bowler, overs, SUM(batsman_run) AS Runs_Conceding, 
           COUNT(CASE WHEN isWicketDelivery = 1 THEN 1 ELSE NULL END) AS Wickets,
           SUM(CASE WHEN non_boundary = 0 THEN 1 ELSE 0 END) AS Dots,
           SUM(CASE WHEN batsman_run = 4 THEN 1 ELSE 0 END) AS Four_Count,
           SUM(CASE WHEN batsman_run = 6 THEN 1 ELSE 0 END) AS Six_Count,
           SUM(CASE WHEN extra_type = 'wide' THEN 1 ELSE 0 END) AS Wide,
           SUM(CASE WHEN extra_type = 'noballs' THEN 1 ELSE 0 END) AS noballs_bowled
    FROM ipl_ball_by_ball
    WHERE ID = ? AND innings = ?
    GROUP BY bowler, overs;
    '''
    bowling_data = pd.read_sql_query(bowling_query, conn, params=(ID, innings))

    conn.close()

    return bowling_data

# Example usage:
ID = 1312200
innings = 1

batting_scorecard = fetch_batting_scorecard(ID, innings)
bowling_scorecard = fetch_bowling_scorecard(ID, innings)

batting_scorecard

,batter,Runs_Scored,Balls_Faced,Four_Count,Six_Count,Out,Dismissal_Type,fielders_involved,Bowler
0,JC Buttler,39,35,5,0,1,caught,WP Saha,HH Pandya
1,YBK Jaiswal,22,16,1,2,1,caught,R Sai Kishore,Yash Dayal
2,SV Samson,14,11,2,0,1,caught,R Sai Kishore,HH Pandya
3,D Padikkal,2,10,0,0,1,caught,Mohammed Shami,Rashid Khan
4,SO Hetmyer,11,12,2,0,1,caught and bowled,None,HH Pandya
5,R Ashwin,6,9,0,0,1,caught,DA Miller,R Sai Kishore
6,R Parag,15,15,1,0,2,run out,R Tewatia,Mohammed Shami
7,TA Boult,11,7,0,1,1,caught,R Tewatia,R Sai Kishore
8,OC McCoy,8,5,0,1,0,not out,None,None


In [19]:
bowling_scorecard

,bowler,overs,Runs_Conceding,Wickets,Dots,Four_Count,Six_Count,Wide,noballs_bowled
0,HH Pandya,8,1,1,6,0,0,0,0
1,HH Pandya,10,3,0,6,0,0,0,0
2,HH Pandya,12,3,1,6,0,0,0,0
3,HH Pandya,14,10,1,6,2,0,0,0
4,LH Ferguson,4,6,0,6,1,0,0,0
5,LH Ferguson,6,10,0,6,2,0,0,0
6,LH Ferguson,16,6,0,6,0,0,0,0
7,Mohammed Shami,0,1,0,6,0,0,0,0
8,Mohammed Shami,2,14,0,6,1,1,0,0
9,Mohammed Shami,9,11,0,6,2,0,0,0
